In [10]:
import numpy as np
import pandas as pd
from ast import literal_eval

In [136]:
clues = pd.read_csv('../data/clues.csv')
charades = pd.read_csv('../data/charades.csv').drop(['clue_rowid_label', 'rowid'], axis=1)
indicators = pd.read_csv('../data/indicators.csv').drop(['clue_rowid_label'], axis=1)

In [137]:
clues = clues[clues['clue'].notna()]
clues = clues[clues['answer'].str.len() > 2]
clues = clues[~clues['clue'].str.contains(r'^See [0-9]')]
clues = clues[clues['clue'].str.len() > 3]
clues.shape

(654850, 10)

In [138]:
clues['score'] = 0
clues['puzzle_date'] = pd.to_datetime(clues['puzzle_date'])
clues['date_used'] = pd.to_datetime('1970-01-01')

C:\Users\shubh\AppData\Local\Temp\ipykernel_25400\1877658815.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  clues['puzzle_date'] = pd.to_datetime(clues['puzzle_date'])


In [139]:
clues.to_csv('../data/clues.csv', index=False)

In [39]:
charades = charades.rename({'clue_rowid': 'rowid', 'answer': 'key'}, axis=1)
charades['charade'] = charades.apply(lambda r: {r['charade']: r['key']}, axis=1)
charades = charades.groupby('rowid')['charade'].agg(list).reset_index()
charades.head()

,rowid,charade
0,1,"[{'chap': 'COVE'}, {'as we see it': 'TO US'}]"
1,21,[{'give the thing acclaim': 'LAUD IT'}]
2,30,[{'queen': 'ER'}]
3,31,"[{'share': 'PART'}, {'mistake': 'ERR'}]"
4,32,"[{'concerned with': 'RE'}, {'emergency': 'URGE..."


In [40]:
indicators = indicators.rename({'clue_rowid': 'rowid'}, axis=1)
indicators.head()

,rowid,alternation,anagram,container,deletion,hidden,homophone,insertion,reversal
0,90,NaN,transforming,NaN,NaN,NaN,NaN,NaN,NaN
1,97,NaN,ground,NaN,NaN,NaN,NaN,NaN,NaN
2,101,NaN,NaN,NaN,NaN,NaN,verbally,NaN,NaN
3,142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,about to go back
4,145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,returned


In [50]:
all_data = clues.merge(charades, on='rowid', how='outer').merge(indicators, on='rowid', how='outer')
all_data = all_data[all_data['answer'].notna()]

In [52]:
all_data.head()

,rowid,clue,answer,definition,clue_number,puzzle_date,puzzle_name,source_url,source,charade,alternation,anagram,container,deletion,hidden,homophone,insertion,reversal
0,1,"Acquisitive chap, as we see it (8)",COVETOUS,Acquisitive,1a,2019-08-08,Times 27424,https://times-xwd-times.livejournal.com/218581...,times_xwd_times,"[{'chap': 'COVE'}, {'as we see it': 'TO US'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Back yard fencing weak and sagging (6),DROOPY,sagging,5a,2019-08-08,Times 27424,https://times-xwd-times.livejournal.com/218581...,times_xwd_times,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,"Stripping off uniform, love holding colonel's ...",UNCLOTHING,Stripping,8a,2019-08-08,Times 27424,https://times-xwd-times.livejournal.com/218581...,times_xwd_times,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Without a mark where they should be gained (4),EXAM,where they should be gained,9a,2019-08-08,Times 27424,https://times-xwd-times.livejournal.com/218581...,times_xwd_times,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,"Put a stop to Rugby's foul school leader (5,2,...",KNOCK ON THE HEAD,Put a stop to,10a,2019-08-08,Times 27424,https://times-xwd-times.livejournal.com/218581...,times_xwd_times,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
all_data['source'].value_counts()

bigdave44                  232535
fifteensquared             224337
times_xwd_times            100725
thehinducrosswordcorner     71713
natpostcryptic              11014
cru_cryptics                 7286
nytimes                      4687
newyorker                    3242
thebrowser                   2601
leoedit                       214
Name: source, dtype: int64

In [58]:
all_data[all_data['source'] == 'natpostcryptic']

,rowid,clue,answer,definition,clue_number,puzzle_date,puzzle_name,source_url,source,charade,alternation,anagram,container,deletion,hidden,homophone,insertion,reversal
575155,578195,Performing in the third month's Butterfly (7),MONARCH,Butterfly,1a,2014-03-15,"Saturday, March 15, 2014 — Harsh Language with...",https://natpostcryptic.blogspot.com/2014/03/sa...,natpostcryptic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
575156,578196,Struggle with two pieces of green fruit (7),GRAPPLE,Struggle,5a,2014-03-15,"Saturday, March 15, 2014 — Harsh Language with...",https://natpostcryptic.blogspot.com/2014/03/sa...,natpostcryptic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
575157,578197,Tourist rigs sheet afresh (9),SIGHTSEER,Tourist,9a,2014-03-15,"Saturday, March 15, 2014 — Harsh Language with...",https://natpostcryptic.blogspot.com/2014/03/sa...,natpostcryptic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
575158,578198,Caught the odour of fish (5),SMELT,Caught the odour of/fish,10a,2014-03-15,"Saturday, March 15, 2014 — Harsh Language with...",https://natpostcryptic.blogspot.com/2014/03/sa...,natpostcryptic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
575159,578199,Medical trainee in with the heartless nurse (6),INTERN,Medical trainee,11a,2014-03-15,"Saturday, March 15, 2014 — Harsh Language with...",https://natpostcryptic.blogspot.com/2014/03/sa...,natpostcryptic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619691,622731,Wrinkly-skinned dog he pairs off (4-3),SHAR PEI,Wrinkly-skinned dog,18d,2022-03-12,"Saturday, March 12, 2022 — All Greek To Me (NP...",https://natpostcryptic.blogspot.com/2022/03/sa...,natpostcryptic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
619692,622732,Greek character holding a “forbidden dance” (7),LAMBADA,forbidden dance,19d,2022-03-12,"Saturday, March 12, 2022 — All Greek To Me (NP...",https://natpostcryptic.blogspot.com/2022/03/sa...,natpostcryptic,"[{'key': 'ESC'}, {'objective': 'END'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
619693,622733,Want to get rid of Dad? (6),DESIRE,Want,20d,2022-03-12,"Saturday, March 12, 2022 — All Greek To Me (NP...",https://natpostcryptic.blogspot.com/2022/03/sa...,natpostcryptic,[{'ready': 'RIPE'}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
619694,622734,First of developments above British port (5),DOVER,British port,23d,2022-03-12,"Saturday, March 12, 2022 — All Greek To Me (NP...",https://natpostcryptic.blogspot.com/2022/03/sa...,natpostcryptic,"[{'receptacle': 'BIN'}, {'large': 'BIG'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
